#### Importing Libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import csv

#### Importing Data

In [ ]:
raw_data = open('data.csv', 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
data = list(reader)
data = np.array(data)
data = data[1:, 2:].astype(float)
rows, columns = np.shape(data)
sqrt_col = int(np.ceil(np.sqrt(columns)))
processed_data = np.zeros([rows, 256, 256])
data_ = np.zeros([rows, 256*256])

for i in range(rows):
    data[i] = (data[i] - np.mean(data[i]))/np.max(data[i])
    data_[i] = np.append(data[i], np.zeros(256*256 - columns))

    
for i in range(rows):
    for j in range(sqrt_col):
            processed_data[i,j,:] = data_[i, j * 256 : (j+1)*256]

raw_labels = open('labels.csv', 'rt')
reader_l = csv.reader(raw_labels, delimiter=',', quoting=csv.QUOTE_NONE)
labels = list(reader_l)
labels = np.array(labels)
labels = labels[1:, 1:]

#### Preprocessing Labels

In [ ]:
gene_labels = []
numeric_labels = np.zeros(np.shape(labels))
for i in range(np.shape(labels)[0]):
    if labels[i][0] not in gene_labels:
        gene_labels.append(labels[i][0])
    numeric_labels[i][0] = gene_labels.index(labels[i][0])
numeric_labels = numeric_labels.astype(float)
print(np.shape(gene_labels))

#### Variational Autoencoder

In [ ]:
def sampling(params):
    
    mean, log_variance = params
    batch = tf.keras.backend.shape(mean)[0]
    x = tf.keras.backend.shape(mean)[1]
    y = tf.keras.backend.shape(mean)[2]
    z = tf.keras.backend.shape(mean)[3]
    epsilon = tf.keras.backend.random_normal(shape=(batch, x,y,z))
    
    return mean + tf.keras.backend.exp(0.5 * log_variance) * epsilon
    

In [ ]:
# Defining the Input
Input_1= tf.keras.layers.Input(shape=(256, 256, 1,))
Input_2 = tf.keras.layers.Input(shape = (5,))

# Autoencoder 

## Endoder
enc_conv1 = tf.keras.layers.Conv2D(64, (10,10), activation='relu', input_shape=(256, 256, 1,),kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same' )(Input_1)
enc_norm1 = tf.keras.layers.BatchNormalization()(enc_conv1)
enc_max1 = tf.keras.layers.MaxPooling2D()(enc_norm1)
enc_conv2 = tf.keras.layers.Conv2D(128, (7,7), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_max1)
enc_norm2 = tf.keras.layers.BatchNormalization()(enc_conv2)
enc_max2 = tf.keras.layers.MaxPooling2D()(enc_norm2)
enc_conv3 = tf.keras.layers.Conv2D(128, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_max2)
enc_norm3 = tf.keras.layers.BatchNormalization()(enc_conv3)
enc_max3 = tf.keras.layers.MaxPooling2D()(enc_norm3)
enc_conv4 = tf.keras.layers.Conv2D(32, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_max3)
enc_norm4 = tf.keras.layers.BatchNormalization()(enc_conv4)

## Encoded Conv-Layer
latent_conv = tf.keras.layers.Conv2D(32, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_conv4)
mean = tf.keras.layers.Conv2D(32, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(latent_conv)
log_variance = tf.keras.layers.Conv2D(32, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(latent_conv)

##Sampled
sampled = tf.keras.layers.Lambda(sampling, output_shape=(32,32,32,), name='sampled')([mean, log_variance])

## Decoder 
dec_conv1 = tf.keras.layers.Conv2D(32, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(sampled)
dec_norm1 = tf.keras.layers.BatchNormalization()(dec_conv1)
dec_upsamp1 = tf.keras.layers.UpSampling2D()(dec_norm1)
dec_conv2 = tf.keras.layers.Conv2D(128, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(dec_upsamp1)
dec_norm2 = tf.keras.layers.BatchNormalization()(dec_conv2)
dec_upsamp2 = tf.keras.layers.UpSampling2D()(dec_norm2)
dec_conv3 = tf.keras.layers.Conv2D(128, (7,7), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(dec_upsamp2)
dec_norm3 = tf.keras.layers.BatchNormalization()(dec_conv3)
dec_upsamp3 = tf.keras.layers.UpSampling2D()(dec_norm3)
dec_conv4 = tf.keras.layers.Conv2D(64, (10,10), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(dec_upsamp3)
dec_norm4 = tf.keras.layers.BatchNormalization()(dec_conv4)

### Output Autoencoder
variac_output = tf.keras.layers.Conv2D(1, (10,10), activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(dec_norm4)

### Classifier
flatten = tf.keras.layers.Flatten()(sampled)
classifier = tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4))(flatten)
classifier = tf.keras.layers.Dense(32, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4))(classifier)
classifier_output = tf.keras.layers.Dense(5, activation='softmax')(classifier)

# Compiling the Model
vae = tf.keras.models.Model(inputs=[Input_1, Input_2], outputs = [variac_output, classifier_output])

reconstruction_loss = tf.keras.backend.sum(tf.keras.losses.mse(vae.inputs[0], vae.outputs[0]))
classifier_loss = tf.keras.backend.sum(tf.keras.losses.categorical_crossentropy(vae.inputs[1], vae.outputs[1]))
kl_loss = 1 + log_variance - tf.keras.backend.square(mean) - tf.keras.backend.exp(log_variance)
kl_loss = tf.keras.backend.sum(kl_loss)

loss =  reconstruction_loss + classifier_loss - kl_loss

vae.add_loss(loss)
vae.compile(optimizer='adam')

In [ ]:
vae.summary()

#### Training

In [ ]:
# Data Training
X_train = processed_data
X_train = np.expand_dims(processed_data, axis = 3)
X_labels = tf.keras.utils.to_categorical(numeric_labels)

# Training 
vae.fit([X_train, X_labels], epochs = 10, batch_size = 50)
